# Applied Programming Coding Challenge #2

# General Information

In [ ]:
import torch
from torchvision import datasets, transforms

# 0 Initialize device

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 1 Load data

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))])

train_dataset = datasets.MNIST(root='../data/mnist_data/', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='../data/mnist_data/', train=False, transform=transform, download=False)